https://algs4.cs.princeton.edu/24pq/

In [17]:
debugging = False
#debugging = True
debug2 = False

logging = True

def dbg(f, *args):
    if debugging:
        print(('  DBG:' + f).format(*args))

def dbg_cont(f, *args):
    if debugging:
        print(('  DBG:' + f).format(*args), end='')

def dbg2(f, *args):
    if debug2:
        print((' -DBG2:' + f).format(*args))
        
def log(f, *args):
    if logging:
        print((f).format(*args))
        
def logError(f, *args):
    if logging:
        print(('*** ERROR:' + f).format(*args))
        
def className(instance):
    return type(instance).__name__

def isstr(obj): 
    return isinstance(obj, basestring)

In [18]:
class Stack:
    ''' a simple stack based on lists '''
    def __init__(self):
         self.items = []

    def is_empty(self):
         return len(self.items) == 0

    def push(self, item):
         self.items.append(item)

    def pop(self):
         return self.items.pop()

In [163]:
import re
WORD_RE = re.compile(r'\W*(\w*)\W*')

class Heap(object):
    ''' This is an unordered array representation in that while the top item 
        is always the min or max value (as determined by the constructor), the
        subsequent entries are NOT guarenteed to be in that order. As each item
        is removed from the top, the tree is reordered just enough to place the
        next entry on top.
    '''
    def __init__(self, max=False, comparator=None):
        self.__heap = []
        self.__maxQ = max
        self.__comparator = comparator if comparator else None
        
    def copy(self):
        copy = Heap()
        copy.__heap = self.__heap.copy()
        copy.__maxQ = self.__maxQ
        copy.__comparator = self.__comparator
        return copy

    def size(self):
        ''' the number of items in the heap '''
        return len(self.__heap)

    def __pop(self):
        ''' removes an item from the end of heap storage; frees space '''
        return self.__heap.pop()
    
    def __push(self, val):
        ''' appends an item to the end of heap storage; allocates space '''
        self.__heap.append(val)
    
    def __get(self, i):
        ''' provide 1 based array access '''
        return self.__heap[i-1]

    def __put(self, i, val):
        ''' provide 1 based array storage '''
        self.__heap[i-1] = val
        
    def __less(self, i, j):
        a = self.__get(i)
        b = self.__get(j)
        if self.__comparator:
            return self.__comparator(a, b)
        else:
            return a > b if self.__maxQ else a < b
        
    def __swap(self, i, j):
        a = self.__get(i)
        b = self.__get(j)
        self.__put(i, b)
        self.__put(j, a)
        
    def __swim(self, k):
        ''' swap the item up the heap to its natural location '''
        while k > 1 and self.__less(k, k // 2):
            self.__swap(k, k // 2);
            k = k // 2;

    def __sink(self, j):
        ''' swap the item down into the heap '''
        N = self.size()
        while 2*j <= N:
            k = 2*j
            # Compare to the right node if it's smaller and present.
            if k < N and self.__less(k+1, k): k += 1
            if self.__less(j, k): break
            self.__swap(j, k)
            j = k
            
    def is_empty(self):
        return self.size() == 0

    def peek(self):
        return self.__get(1)
    
    def remove(self):
        ''' Return the top item from the heap '''
        result = self.peek()
        endval = self.__pop()
        N =  N = self.size()
        if N > 0:
            self.__put(1, endval)
            if N > 1: self.__sink(1)
        return result

    def insert(self, val):
        ''' Insert a value into heap '''
        self.__push(val)
        N =  N = self.size()
        self.__swim(N)
        return self # chaining support.
    
    def load(self, alist):
        for x in alist: self.insert(x)
        return self # chaining support.

    def load_word(self, raw_word):
        lword = raw_word.lower()
        m = WORD_RE.match(lword)
        #log('- Match "{0}" for {1}', lword, m)
        word = m.group(1)       
        if len(word):
            #log('- Insert "{0}"', word)
            self.insert(word)
        return self # chaining support.
    
    def load_words(self, *sentences):        
        for sentence in sentences:
            for word in sentence.split():
                self.load_word(word)
        return self # chaining support.
    
    def consume(self, preserve=False):
        ''' Iterate and consume all the items in order '''
        if preserve: self = self.copy()
        while not self.is_empty(): yield self.pop()

    def __iter__(self):
        ''' Iterate all the items, but not necessarily in order '''
        return iter(self.__heap)
        
    def __repr__(self):
        return str([x for x in self])

    __str__ = __repr__
    pop = remove # Convenient and often expected.

In [ ]:
x = Heap(max=False).load([4, 5, 1, 11, 2, 12, 11, 31, 2, 3, 4, 9, 6, 0, -1])
x

In [164]:
[v for v in x.consume()]

[-1, 0, 1, 2, 2, 3, 4, 4, 5, 6, 9, 11, 11, 12, 31]

In [165]:
y=Heap().load_word('%@, Hello  Eat 324')

In [166]:
y=Heap(max=True)
y.load_words("The quick brown fox jumped over the lazy dog.",
                   "Who knows what evil lurks in the hearts of men?",
                   "The shadow knows. ",
                   "Ask not, what your country can do for you.",
                   "Ask what you can do for your country")

['your', 'your', 'you', 'you', 'the', 'what', 'who', 'the', 'what', 'shadow', 'knows', 'the', 'what', 'lurks', 'over', 'lazy', 'the', 'for', 'of', 'jumped', 'quick', 'knows', 'ask', 'brown', 'not', 'evil', 'country', 'can', 'do', 'for', 'in', 'ask', 'fox', 'hearts', 'can', 'do', 'dog', 'men', 'country']

In [167]:
[v for v in y.consume(preserve=True)]

['your',
 'your',
 'you',
 'you',
 'who',
 'what',
 'what',
 'what',
 'the',
 'the',
 'the',
 'the',
 'shadow',
 'quick',
 'over',
 'of',
 'not',
 'men',
 'lurks',
 'lazy',
 'knows',
 'knows',
 'jumped',
 'in',
 'hearts',
 'fox',
 'for',
 'for',
 'evil',
 'dog',
 'do',
 'do',
 'country',
 'country',
 'can',
 'can',
 'brown',
 'ask',
 'ask']

In [172]:
y

['your', 'your', 'you', 'you', 'the', 'what', 'who', 'the', 'what', 'shadow', 'knows', 'the', 'what', 'lurks', 'over', 'lazy', 'the', 'for', 'of', 'jumped', 'quick', 'knows', 'ask', 'brown', 'not', 'evil', 'country', 'can', 'do', 'for', 'in', 'ask', 'fox', 'hearts', 'can', 'do', 'dog', 'men', 'country']

In [173]:
i = y.consume()
next(i)

'your'

In [174]:
next(i)

'your'

In [175]:
y

['you', 'what', 'you', 'the', 'the', 'what', 'who', 'the', 'of', 'shadow', 'knows', 'the', 'what', 'lurks', 'over', 'lazy', 'hearts', 'for', 'men', 'jumped', 'quick', 'knows', 'ask', 'brown', 'not', 'evil', 'country', 'can', 'do', 'for', 'in', 'ask', 'fox', 'country', 'can', 'do', 'dog']

In [176]:
next(i)

'you'